# Project 3: Data Cleaning - Tidy up messy Datasets (Movies Dataset)

1. __Load__ and __inspect__ the messy dataset __movies_metadata.csv__. Identify columns with nested / stringified json data.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("movies_metadata.csv")

C:\Users\Tris\AppData\Local\Temp\ipykernel_10552\1650877637.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("movies_metadata.csv")


In [3]:
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


## Dropping irrelevant Columns

2. __Drop__ the irrelevant columns 'adult', 'imdb_id', 'original_title', 'video' and 'homepage'.

In [4]:
df.drop(["adult",'imdb_id', 'original_title', 'video','homepage'], axis=1, inplace=True)

In [5]:
df.belongs_to_collection[0]

"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}"

## How to handle stringified JSON columns

3. __Evaluate__ Python Expressions in the stringified columns ["belongs_to_collection", "genres", "production_countries", "production_companies", "spoken_languages"] and __remove quotes__ ("") where possible.

In [6]:
import json
import ast
import numpy as np

In [7]:
df.genres = df.genres.apply(ast.literal_eval)

In [8]:
df.belongs_to_collection = df.belongs_to_collection.apply(lambda x: ast.literal_eval(x) if isinstance(x,str) else np.nan)

In [9]:
df.production_countries = df.production_countries.apply(lambda x: ast.literal_eval(x) if isinstance(x,str) else np.nan)
df.production_companies = df.production_companies.apply(lambda x: ast.literal_eval(x) if isinstance(x,str) else np.nan)
df.spoken_languages = df.spoken_languages.apply(lambda x: ast.literal_eval(x) if isinstance(x,str) else np.nan)

In [10]:
df.belongs_to_collection[0]

{'id': 10194,
 'name': 'Toy Story Collection',
 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg',
 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}

In [11]:
df.belongs_to_collection[0]['name']

'Toy Story Collection'

## How to flatten nested Columns

4. __Extract__ only the __collection name__ from the column "belongs_to_collection" and __overwrite__ "belongs_to_collection". <br> For example: The value in the first row (Toy Story) should be 'Toy Story Collection'.

In [12]:
df.belongs_to_collection = df.belongs_to_collection.apply(lambda x: x['name'] if isinstance(x,dict) else np.nan)

In [13]:
df.belongs_to_collection

0                  Toy Story Collection
1                                   NaN
2             Grumpy Old Men Collection
3                                   NaN
4        Father of the Bride Collection
                      ...              
45461                               NaN
45462                               NaN
45463                               NaN
45464                               NaN
45465                               NaN
Name: belongs_to_collection, Length: 45466, dtype: object

5. __Extract__ all __genre names__ from the column "genres" and __overwrite__ "genres". If a movie has more than one genre, __seperate genres by a pipe__ "|".<br>
For example: The value in the first row (Toy Story) should be 'Animation|Comedy|Family'.

In [14]:
df.genres[0][0]

{'id': 16, 'name': 'Animation'}

In [15]:
df.genres = df.genres.apply(lambda x: "|".join(i['name'] for i in x))

In [16]:
df.genres.replace("",np.nan,inplace=True)

6. __Extract__ all __spoken language names__ from the column "spoken_languages" and __overwrite__ "spoken_languages". If a movie has more than one spoken language, __seperate spoken languages by a pipe__ "|".<br>
For example: The value in the first row (Toy Story) should be 'English'.

In [17]:
df.spoken_languages[0]

[{'iso_639_1': 'en', 'name': 'English'}]

In [18]:
df.spoken_languages = df.spoken_languages.apply(lambda x: "|".join(i['name'] for i in x) if isinstance(x,list) else np.nan)

In [19]:
df.spoken_languages.replace("", np.nan, inplace = True)

7. __Extract__ all __production countries names__ from the column "production_countries" and __overwrite__ "production_countries". If a movie has more than one production country, __seperate production countries by a pipe__ "|".<br>
For example: The value in the first row (Toy Story) should be 'United States of America'.

In [20]:
df.production_countries = df.production_countries.apply(lambda x: "|".join(i['name'] for i in x) if isinstance(x,list) else np.nan)

In [21]:
df.production_countries.replace("", np.nan, inplace = True)

8. __Extract__ all __production companies names__ from the column "production_companies" and __overwrite__ "production_companies". If a movie has more than one production company, __seperate production companies by a pipe__ "|".<br>
For example: The value in the first row (Toy Story) should be 'Pixar Animation Studios'

In [22]:
df.production_companies = df.production_companies.apply(lambda x: "|".join(i['name'] for i in x) if isinstance(x,list) else np.nan)

In [23]:
df.production_companies.replace("", np.nan, inplace = True)

9. __Inspect__ all columns above with value_counts(). Do you see anything strange? __Take reasonable measures__!

In [24]:
df.isnull().sum()

belongs_to_collection    40975
budget                       0
genres                    2442
id                           0
original_language           11
overview                   954
popularity                   5
poster_path                386
production_companies     11881
production_countries      6288
release_date                87
revenue                      6
runtime                    263
spoken_languages          3958
status                      87
tagline                  25054
title                        6
vote_average                 6
vote_count                   6
dtype: int64

## Cleaning Numerical Columns

10. __Convert__ the datatype in the columns __"budget"__, __"id"__ and __"popularity"__ __to numeric__. Set invalid values as NaN.

In [25]:
df.budget = pd.to_numeric(df.budget, errors = "coerce")

In [26]:
df.budget.replace(0,np.nan, inplace=True)

In [27]:
df.budget = df.budget.div(1000000)

In [28]:
df.id = pd.to_numeric(df.id, errors = "coerce")
df.popularity = pd.to_numeric(df.popularity, errors = "coerce")

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4491 non-null   object 
 1   budget                 8890 non-null   float64
 2   genres                 43024 non-null  object 
 3   id                     45463 non-null  float64
 4   original_language      45455 non-null  object 
 5   overview               44512 non-null  object 
 6   popularity             45460 non-null  float64
 7   poster_path            45080 non-null  object 
 8   production_companies   33585 non-null  object 
 9   production_countries   39178 non-null  object 
 10  release_date           45379 non-null  object 
 11  revenue                45460 non-null  float64
 12  runtime                45203 non-null  float64
 13  spoken_languages       41508 non-null  object 
 14  status                 45379 non-null  object 
 15  ta

11. __Analyze__ the columns __"budget"__ and __"revenue"__ and __"runtime"__. Analyze movies with a budget/revenue/runtime of 0. Do you think the value 0 is the most appropriate value? __Take reasonable measures__! 

In [30]:
df.revenue.replace(0,np.nan, inplace=True)

In [31]:
df.revenue = df.revenue.div(1000000)

In [32]:
df.runtime.replace(0,np.nan, inplace=True)

In [33]:
df.runtime.value_counts(dropna=False)

90.0     2556
NaN      1821
100.0    1470
95.0     1412
93.0     1214
         ... 
410.0       1
283.0       1
238.0       1
566.0       1
780.0       1
Name: runtime, Length: 353, dtype: int64

In [34]:
df.rename(columns = {"revenue": "revenue_musd", "budget": "budget_musd"}, inplace=True)

13. __Analyze__ movies with a __vote_count of 0__. What´s the __vote_average__ for those movies? Do you think this value is the most appropriate value? __Take reasonable measures__!

In [35]:
df.loc[df.vote_count == 0, "vote_average"] = np.nan

## Cleaning DateTime Columns

14. __Convert__ the datatype in the column __"release_date"__ __to datetime__. Set invalid values as NaN.

In [36]:
pd.to_datetime(df.release_date, errors = "coerce")

0       1995-10-30
1       1995-12-15
2       1995-12-22
3       1995-12-22
4       1995-02-10
           ...    
45461          NaT
45462   2011-11-17
45463   2003-08-01
45464   1917-10-21
45465   2017-06-09
Name: release_date, Length: 45466, dtype: datetime64[ns]

## Cleaning Text / String Columns

15. __Analyze__ the text columns "overview" and "tagline". Try to identify __missing data that is not represented by NaN__ (e.g. "No Data"). __Replace as NaN__ (np.nan)!

In [37]:
df.overview.value_counts(dropna=False)

NaN                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      954
No overview found.                                                                                                                                                                                                                         

In [38]:
df.overview.replace(" ", np.nan, inplace=True)
df.overview.replace("No overview found.", np.nan, inplace=True)
df.overview.replace("No Overview", np.nan, inplace=True)
df.overview.replace("No movie overview available.", np.nan, inplace=True)
df.overview.replace("No overview yet.", np.nan, inplace=True)

In [39]:
df.tagline.replace("-", np.nan, inplace=True)

## Removing Duplicates

16. __Identify__ and __remove__ duplicates!

In [40]:
df.duplicated(keep = False).value_counts()

False    45433
True        33
dtype: int64

__There are 33 duplicates__

In [41]:
df.drop_duplicates(inplace=True)

In [42]:
df.duplicated(subset = "id",keep = False).value_counts()

False    45420
True        29
dtype: int64

__There are 29 duplicates of id column__

In [43]:
df.drop_duplicates(subset = "id", inplace=True)

In [44]:
df.id.value_counts(dropna=False)

862.0       1
74458.0     1
296206.0    1
107308.0    1
16247.0     1
           ..
44399.0     1
10138.0     1
32084.0     1
42191.0     1
461257.0    1
Name: id, Length: 45434, dtype: int64

## Handling Missing Values & Removing Observations

17. __Drop__ all rows/movies with unknown __id__ or __title__.

In [45]:
df.isna().sum()

belongs_to_collection    40946
budget_musd              36554
genres                    2442
id                           1
original_language           11
overview                  1104
popularity                   4
poster_path                386
production_companies     11872
production_countries      6283
release_date                87
revenue_musd             38036
runtime                   1819
spoken_languages          3954
status                      85
tagline                  25037
title                        4
vote_average              2900
vote_count                   4
dtype: int64

In [46]:
df.dropna(subset= ["title", "id"],inplace = True)

In [47]:
df.id = df.id.astype("int")

In [48]:
df.notna().sum(axis=1).value_counts()

15    12522
16    11454
14     5424
17     4265
18     3859
13     3040
12     1891
19     1132
11     1020
10      511
9       184
8       104
7        20
6         4
dtype: int64

In [49]:
df[df.notna().sum(axis=1) == 6]

,belongs_to_collection,budget_musd,genres,id,original_language,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue_musd,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
18038,NaN,NaN,NaN,344741,pl,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Released,NaN,Pajęczarki,NaN,0.0
20166,NaN,NaN,NaN,139909,fi,NaN,0.000127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Released,NaN,Aurinkotuuli,NaN,0.0
41718,NaN,NaN,NaN,216550,de,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Released,NaN,Dolpo Tulku - Heimkehr in den Himalaya,NaN,0.0
44978,NaN,NaN,NaN,398295,pt,NaN,0.000331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Released,NaN,Mundo Cão,NaN,0.0


18. __Keep__ only those rows/movies in the df with __10 or more non-NaN__ values.

In [50]:
df.dropna(thresh=10, inplace=True)

## Final (Cleaning) Steps

19. __Keep__ only those rows/movies in the df with __status "Released"__. Then __drop__ the column "status".

In [51]:
df.status.value_counts()

Released           44691
Rumored              226
Post Production       98
In Production         20
Planned               15
Canceled               2
Name: status, dtype: int64

In [52]:
df = df.loc[df.status == "Released"].copy()

In [53]:
df.drop(columns = "status", inplace = True)

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44691 entries, 0 to 45465
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4463 non-null   object 
 1   budget_musd            8854 non-null   float64
 2   genres                 42586 non-null  object 
 3   id                     44691 non-null  int32  
 4   original_language      44681 non-null  object 
 5   overview               43740 non-null  object 
 6   popularity             44691 non-null  float64
 7   poster_path            44467 non-null  object 
 8   production_companies   33356 non-null  object 
 9   production_countries   38835 non-null  object 
 10  release_date           44657 non-null  object 
 11  revenue_musd           7385 non-null   float64
 12  runtime                43179 non-null  float64
 13  spoken_languages       41094 non-null  object 
 14  tagline                20284 non-null  object 
 15  ti

20. The Order of the columns should be as follows: 

In [56]:
col = ["id", "title", "tagline", "release_date", "genres", "belongs_to_collection", 
"original_language", "budget_musd", "revenue_musd", "production_companies",
"production_countries", "vote_count", "vote_average", "popularity", "runtime",
"overview", "spoken_languages", "poster_path"]

In [57]:
df = df.loc[:, col]

In [58]:
df

,id,title,tagline,release_date,genres,belongs_to_collection,original_language,budget_musd,revenue_musd,production_companies,production_countries,vote_count,vote_average,popularity,runtime,overview,spoken_languages,poster_path
0,862,Toy Story,NaN,1995-10-30,Animation|Comedy|Family,Toy Story Collection,en,30.0,373.554033,Pixar Animation Studios,United States of America,5415.0,7.7,21.946943,81.0,"Led by Woody, Andy's toys live happily in his ...",English,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg
1,8844,Jumanji,Roll the dice and unleash the excitement!,1995-12-15,Adventure|Fantasy|Family,NaN,en,65.0,262.797249,TriStar Pictures|Teitler Film|Interscope Commu...,United States of America,2413.0,6.9,17.015539,104.0,When siblings Judy and Peter discover an encha...,English|Français,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg
2,15602,Grumpier Old Men,Still Yelling. Still Fighting. Still Ready for...,1995-12-22,Romance|Comedy,Grumpy Old Men Collection,en,NaN,NaN,Warner Bros.|Lancaster Gate,United States of America,92.0,6.5,11.712900,101.0,A family wedding reignites the ancient feud be...,English,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg
3,31357,Waiting to Exhale,Friends are the people who let you be yourself...,1995-12-22,Comedy|Drama|Romance,NaN,en,16.0,81.452156,Twentieth Century Fox Film Corporation,United States of America,34.0,6.1,3.859495,127.0,"Cheated on, mistreated and stepped on, the wom...",English,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg
4,11862,Father of the Bride Part II,Just When His World Is Back To Normal... He's ...,1995-02-10,Comedy,Father of the Bride Collection,en,NaN,76.578911,Sandollar Productions|Touchstone Pictures,United States of America,173.0,5.7,8.387519,106.0,Just when George Banks has recovered from his ...,English,/e64sOI48hQXyru7naBFyssKFxVd.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,439050,Subdue,Rising and falling between a man and woman,NaN,Drama|Family,NaN,fa,NaN,NaN,NaN,Iran,1.0,4.0,0.072051,90.0,Rising and falling between a man and woman.,فارسی,/jldsYflnId4tTWPx8es3uzsB1I8.jpg
45462,111109,Century of Birthing,NaN,2011-11-17,Drama,NaN,tl,NaN,NaN,Sine Olivia,Philippines,3.0,9.0,0.178241,360.0,An artist struggles to finish his work while a...,NaN,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg
45463,67758,Betrayal,A deadly game of wits.,2003-08-01,Action|Drama|Thriller,NaN,en,NaN,NaN,American World Pictures,United States of America,6.0,3.8,0.903007,90.0,"When one of her hits goes wrong, a professiona...",English,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg
45464,227506,Satan Triumphant,NaN,1917-10-21,NaN,NaN,en,NaN,NaN,Yermoliev,Russia,0.0,NaN,0.003503,87.0,"In a small town live two brothers, one a minis...",NaN,/aorBPO7ak8e8iJKT5OcqYxU3jlK.jpg


21. __Reset__ the Index and create a __RangeIndex__.

In [60]:
df.reset_index(drop = True,inplace = True)

In [61]:
df.index = pd.RangeIndex(start=0, stop=len(df.index), step=1)

22. __Save__ the cleaned dataset in a __csv-file__.

In [66]:
df.to_csv("clean_data.csv", index =  False)